# Heiko NMR Transceiver Configuration Notebook
# Author Yichao Peng

### Import Bitstream

In [1]:
from pynq import Overlay
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
ol = Overlay('HVNMR.bit') # high voltage NMR

In [2]:
# ol?
chip_conf = ol.fpga_nmr_chip_config_0

### SPI-configuration Command Addresses

In [4]:
# Constants
register_width = 32

# Register 0 and 2 are for SPI data
# Register 1 is for start the transmission
# Register 3 is for read out of done signal

Address_register_SPI_data = tuple(x * 4 for x in (0, 2))
Address_register_start = 1 * 4
Address_register_done = 3 * 4 # 末尾加,为元组

# Sub-adress of SPI configuration under Register 0 and 2, 55 bits in total

sub_addr_array = [
    0,   # Sub_Address_SPI_pll_en
    1,   # Sub_Address_SPI_gain
    2,   # Sub_Address_SPI_skip_mixer
    3,   # Sub_Address_SPI_prescaler_pll
    5,   # Sub_Address_SPI_N_divider_pll
    10,  # Sub_Address_SPI_prescaler_tx_logic
    12,  # Sub_Address_SPI_tx_shortening_counter
    20,  # Sub_Address_SPI_deadtime_hs_p
    24,  # Sub_Address_SPI_deadtime_ls_p
    28,  # Sub_Address_SPI_deadtime_hs_n
    32,  # Sub_Address_SPI_deadtime_ls_n
    36,  # Sub_Address_SPI_deadtime_comp_p
    40,  # Sub_Address_SPI_deadtime_comp_n
    44,  # Sub_Address_SPI_delay_p
    48,  # Sub_Address_SPI_delay_n
    52,  # Sub_Address_SPI_amplifier_reset
    53,  # Sub_Address_SPI_pll_or_spi_output
    54   # Sub_Address_SPI_ls
]

# 18 SPI configuration commands in total

Sub_Address_SPI_pll_en = 0
Sub_Address_SPI_gain = 1
Sub_Address_SPI_skip_mixer = 2
Sub_Address_SPI_prescaler_pll = 3
Sub_Address_SPI_N_divider_pll = 4
Sub_Address_SPI_prescaler_tx_logic = 5
Sub_Address_SPI_tx_shortening_counter = 6
Sub_Address_SPI_deadtime_hs_p = 7
Sub_Address_SPI_deadtime_ls_p = 8
Sub_Address_SPI_deadtime_hs_n = 9
Sub_Address_SPI_deadtime_ls_n = 10
Sub_Address_SPI_deadtime_comp_p = 11
Sub_Address_SPI_deadtime_comp_n = 12
Sub_Address_SPI_delay_p = 13
Sub_Address_SPI_delay_n = 14
Sub_Address_SPI_amplifier_reset = 15
Sub_Address_SPI_pll_or_spi_output = 16
Sub_Address_SPI_ls = 17

Basic functions, just run it

In [5]:
# Basic functions

# Read whole register value
def read_register(address):
    return chip_conf.read(address)

# Write whole register value
def write_register(address, val):
    return chip_conf.write(address, val)

# Clear all registers
def clear_register():
    chip_conf.write(Address_register_SPI_data[0], 0)
    chip_conf.write(Address_register_SPI_data[1], 0)
    chip_conf.write(Address_register_start, 0)
    chip_conf.write(Address_register_done, 0)

# Read SPI data register sub-function value
def read_spi_sub_address(sub_address):
    cur = chip_conf.read(Address_register_SPI_data[1]) + 2**32 * chip_conf.read(Address_register_SPI_data[0])
    if (sub_address == 17):
        len = 1
    else:
        len = sub_addr_array[sub_address + 1] - sub_addr_array[sub_address]
    result = (cur >> (9 + sub_addr_array[sub_address])) % (2**len)
    return result
    
# Read SPI data register specific bit value
def read_bit(address, bit):
    pass
    
# Write SPI data register sub-function value
def write_spi_sub_address(sub_address, set_val):
    address = Address_register_SPI_data[0] if sub_addr_array[sub_address] > 22 else Address_register_SPI_data[1]  # firstly find out whether the subaddress in register 0 or 2
    cur_0 = read_register(Address_register_SPI_data[0])
    cur_2 = read_register(Address_register_SPI_data[1])
    if (sub_address == Sub_Address_SPI_deadtime_hs_p): # special, need change both register 0 and 2
        print("Command in both register 0 and register 2.")
        print(4)
        if (set_val >= 8):
            masked = cur_0 & (2 ** register_width - 1 - 1)
            result_0 = masked | 1
            set_val = set_val - 8
            # print(set_val)
        else:
            masked = cur_0 & (2 ** register_width - 1 - 1)
            result_0 = masked
        chip_conf.write(Address_register_SPI_data[0], result_0)
        max_val = 7
        set_val = set_val if (set_val<=max_val) else max_val
        print(set_val)
        masked = cur_2 & (2 ** register_width - 1 - (max_val << 29))
        result_2 = masked | (set_val << 29)
        chip_conf.write(Address_register_SPI_data[1], result_2)
    else:
        if (sub_address == 17):
            len = 1
        else:
            len = sub_addr_array[sub_address + 1] - sub_addr_array[sub_address]
        print(len)
        max_val = 2 ** len - 1
        if (set_val > max_val):
            set_val = max_val
        if (address == Address_register_SPI_data[1]):
            masked = cur_2 & (2 ** register_width - 1 - (max_val << (sub_addr_array[sub_address] + 9)))
            result_2 = masked | (set_val << (sub_addr_array[sub_address] + 9))
            chip_conf.write(Address_register_SPI_data[1], result_2)
        else:
            masked = cur_0 & (2 ** register_width - 1 - (max_val << (sub_addr_array[sub_address] - 23)))
            result_0 = masked | (set_val << (sub_addr_array[sub_address] - 23))
            chip_conf.write(Address_register_SPI_data[0], result_0)
    return

##### Register 1

In [6]:
def spi_write_start(val): # rising edge trigger
    chip_conf.write(Address_register_start, val)

##### Register 3

In [7]:
def spi_read_finish(): # spi finish signal
    chip_conf.read(Address_register_done)

##### Register 0 and 2

In [8]:
# Bit 0: internal PLL
def spi_read_pll_en():
    return read_spi_sub_address(Sub_Address_SPI_pll_en)
def spi_write_pll_en(val):
    return write_spi_sub_address(Sub_Address_SPI_pll_en, val)

# Bit 1: gain
def spi_read_gain():
    return read_spi_sub_address(Sub_Address_SPI_gain)
def spi_write_gain(val):
    return write_spi_sub_address(Sub_Address_SPI_gain, val)

# Bit 2: skip_mixer
def spi_read_skip_mixer():
    return read_spi_sub_address(Sub_Address_SPI_skip_mixer)
def spi_write_skip_mixer(val):
    return write_spi_sub_address(Sub_Address_SPI_skip_mixer, val)

# Bit 3-4: prescaler_pll
def spi_read_prescaler_pll():
    return read_spi_sub_address(Sub_Address_SPI_prescaler_pll)
def spi_write_prescaler_pll(val):
    return write_spi_sub_address(Sub_Address_SPI_prescaler_pll, val)

# Bit 5-9: N_divider_pll
def spi_read_N_divider_pll():
    return read_spi_sub_address(Sub_Address_SPI_N_divider_pll)
def spi_write_N_divider_pll(val):
    return write_spi_sub_address(Sub_Address_SPI_N_divider_pll, val)

# Bit 10-11: prescaler_tx_logic
def spi_read_prescaler_tx_logic():
    return read_spi_sub_address(Sub_Address_SPI_prescaler_tx_logic)
def spi_write_prescaler_tx_logic(val):
    return write_spi_sub_address(Sub_Address_SPI_prescaler_tx_logic, val)

# Bit 12-19: tx_shortening_counter
def spi_read_tx_shortening_counter():
    return read_spi_sub_address(Sub_Address_SPI_tx_shortening_counter)
def spi_write_tx_shortening_counter(val):
    return write_spi_sub_address(Sub_Address_SPI_tx_shortening_counter, val)

# Bit 20-23: deadtime_hs_p
def spi_read_deadtime_hs_p():
    return read_spi_sub_address(Sub_Address_SPI_deadtime_hs_p)
def spi_write_deadtime_hs_p(val):
    return write_spi_sub_address(Sub_Address_SPI_deadtime_hs_p, val)

# Bit 24-27: deadtime_ls_p
def spi_read_deadtime_ls_p():
    return read_spi_sub_address(Sub_Address_SPI_deadtime_ls_p)
def spi_write_deadtime_ls_p(val):
    return write_spi_sub_address(Sub_Address_SPI_deadtime_ls_p, val)

# Bit 28-31: deadtime_hs_n
def spi_read_deadtime_hs_n():
    return read_spi_sub_address(Sub_Address_SPI_deadtime_hs_n)
def spi_write_deadtime_hs_n(val):
    return write_spi_sub_address(Sub_Address_SPI_deadtime_hs_n, val)

# Bit 32-35: deadtime_ls_n
def spi_read_deadtime_ls_n():
    return read_spi_sub_address(Sub_Address_SPI_deadtime_ls_n)
def spi_write_deadtime_ls_n(val):
    return write_spi_sub_address(Sub_Address_SPI_deadtime_ls_n, val)

# Bit 36-39: deadtime_comp_p
def spi_read_deadtime_comp_p():
    return read_spi_sub_address(Sub_Address_SPI_deadtime_comp_p)
def spi_write_deadtime_comp_p(val):
    return write_spi_sub_address(Sub_Address_SPI_deadtime_comp_p, val)

# Bit 40-43: deadtime_comp_n
def spi_read_deadtime_comp_n():
    return read_spi_sub_address(Sub_Address_SPI_deadtime_comp_n)
def spi_write_deadtime_comp_n(val):
    return write_spi_sub_address(Sub_Address_SPI_deadtime_comp_n, val)

# Bit 44-47: delay_p
def spi_read_delay_p():
    return read_spi_sub_address(Sub_Address_SPI_delay_p)
def spi_write_delay_p(val):
    return write_spi_sub_address(Sub_Address_SPI_delay_p, val)

# Bit 48-51: delay_n
def spi_read_delay_n():
    return read_spi_sub_address(Sub_Address_SPI_delay_n)
def spi_write_delay_n(val):
    return write_spi_sub_address(Sub_Address_SPI_delay_n, val)

# Bit 52: amplifier_reset
def spi_read_amplifier_reset():
    return read_spi_sub_address(Sub_Address_SPI_amplifier_reset)
def spi_write_amplifier_reset(val):
    return write_spi_sub_address(Sub_Address_SPI_amplifier_reset, val)

# Bit 53: pll_or_spi_output
def spi_read_pll_or_spi_output():
    return read_spi_sub_address(Sub_Address_SPI_pll_or_spi_output)
def spi_write_pll_or_spi_output(val):
    return write_spi_sub_address(Sub_Address_SPI_pll_or_spi_output, val)

# Bit 54: ls
def spi_read_ls():
    return read_spi_sub_address(Sub_Address_SPI_ls)
def spi_write_ls(val):
    return write_spi_sub_address(Sub_Address_SPI_ls, val)

Clear register, set all values to 0

In [9]:
clear_register()

Set individual commands, type in decimal number, e.g. 111 is 7, 10 is 2

In [37]:
# 0
spi_write_pll_en(1)

1


In [11]:
# 1
spi_write_gain(1)

1


In [12]:
# 2
spi_write_skip_mixer(1)

1


In [13]:
# 3
spi_write_prescaler_pll(3)

2


In [14]:
# 4
spi_write_N_divider_pll(2**5-1)

5


In [15]:
# 5
spi_write_prescaler_tx_logic(3)

2


In [16]:
# 6
spi_write_tx_shortening_counter(2**8-1)

8


In [17]:
# 7
spi_write_deadtime_hs_p(15)

Command in both register 0 and register 2.
4
7


In [18]:
# 8
spi_write_deadtime_ls_p(15)

4


In [39]:
# 9
spi_write_deadtime_hs_n(15)

4


In [20]:
# 10
spi_write_deadtime_ls_n(15)

4


In [21]:
# 11
spi_write_deadtime_comp_p(15)

4


In [22]:
# 12
spi_write_deadtime_comp_n(15)

4


In [23]:
# 13
spi_write_delay_p(15)

4


In [24]:
# 14
spi_write_delay_n(15)

4


In [25]:
# 15
spi_write_pll_or_spi_output(1)

1


In [26]:
# 16
spi_write_amplifier_reset(1)

1


In [32]:
# 17
spi_write_ls(1)

1


Start tranfer SPI commands

In [40]:
spi_write_start(1)